In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from custom_models import cnn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [3]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [4]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [5]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [6]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [7]:
num_epochs = 10
num_sims = 10
train_percs = [0.8]#[1.0, 0.8, 0.8, 0.6, 0.6, 0.4, 0.4]
synth_percs = [0.0]#[0.0, 0.2, 0.0, 0.4, 0.0, 0.6, 0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = cnn.CNN(in_channels=1, num_classes=4)

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            #num_ftrs = 4 #model.fc.in_features
            #model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_cnn_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0536 Acc: 0.4949
test Loss: 1.0038 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0164 Acc: 0.5018
test Loss: 0.9434 Acc: 0.5516
Epoch 2/9
----------
synthetic_train Loss: 0.9853 Acc: 0.5211
test Loss: 0.8737 Acc: 0.5586
Epoch 3/9
----------
synthetic_train Loss: 0.9379 Acc: 0.5389
test Loss: 0.8622 Acc: 0.5664
Epoch 4/9
----------
synthetic_train Loss: 0.8942 Acc: 0.5611
test Loss: 0.7465 Acc: 0.6461
Epoch 5/9
----------
synthetic_train Loss: 0.8484 Acc: 0.5957
test Loss: 0.6493 Acc: 0.7242
Epoch 6/9
----------
synthetic_train Loss: 0.7907 Acc: 0.6357
test Loss: 0.5723 Acc: 0.7719
Epoch 7/9
----------
synthetic_train Loss: 0.7286 Acc: 0.6676
test Loss: 0.5117 Acc: 0.8039
Epoch 8/9
----------
synthetic_train Loss: 0.6766 Acc: 0.6941
test Loss: 0.4718 Acc: 0.8492
Epoch 9/9
----------
synthetic_train Loss: 0.6394 Acc: 0.7119
test Loss: 0.4258 Acc: 0.8523

Training complete in 3m 48s
Best test Loss: 0.425759


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0522 Acc: 0.4998
test Loss: 0.9651 Acc: 0.5750
Epoch 1/9
----------
synthetic_train Loss: 1.0056 Acc: 0.5213
test Loss: 0.9256 Acc: 0.5719
Epoch 2/9
----------
synthetic_train Loss: 0.9783 Acc: 0.5254
test Loss: 0.8077 Acc: 0.6344
Epoch 3/9
----------
synthetic_train Loss: 0.9240 Acc: 0.5633
test Loss: 0.6591 Acc: 0.7086
Epoch 4/9
----------
synthetic_train Loss: 0.8562 Acc: 0.5998
test Loss: 0.5593 Acc: 0.7789
Epoch 5/9
----------
synthetic_train Loss: 0.7713 Acc: 0.6402
test Loss: 0.4880 Acc: 0.8070
Epoch 6/9
----------
synthetic_train Loss: 0.7135 Acc: 0.6723
test Loss: 0.4675 Acc: 0.8391
Epoch 7/9
----------
synthetic_train Loss: 0.6687 Acc: 0.7045
test Loss: 0.3445 Acc: 0.8797
Epoch 8/9
----------
synthetic_train Loss: 0.6149 Acc: 0.7291
test Loss: 0.3494 Acc: 0.8852
Epoch 9/9
----------
synthetic_train Loss: 0.5538 Acc: 0.7623
test Loss: 0.2670 Acc: 0.9062

Training complete in 3m 38s
Best test Loss: 0.267002


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 1.0573 Acc: 0.4965
test Loss: 1.0439 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0331 Acc: 0.5018
test Loss: 0.9195 Acc: 0.5453
Epoch 2/9
----------
synthetic_train Loss: 1.0054 Acc: 0.5207
test Loss: 0.8544 Acc: 0.5711
Epoch 3/9
----------
synthetic_train Loss: 0.9782 Acc: 0.5271
test Loss: 0.8105 Acc: 0.6195
Epoch 4/9
----------
synthetic_train Loss: 0.9424 Acc: 0.5496
test Loss: 0.9063 Acc: 0.6281
Epoch 5/9
----------
synthetic_train Loss: 0.9005 Acc: 0.5646
test Loss: 0.6720 Acc: 0.6992
Epoch 6/9
----------
synthetic_train Loss: 0.8428 Acc: 0.5939
test Loss: 0.6016 Acc: 0.7438
Epoch 7/9
----------
synthetic_train Loss: 0.7921 Acc: 0.6193
test Loss: 0.5244 Acc: 0.8016
Epoch 8/9
----------
synthetic_train Loss: 0.7666 Acc: 0.6404
test Loss: 0.7931 Acc: 0.7094
Epoch 9/9
----------
synthetic_train Loss: 0.7283 Acc: 0.6682
test Loss: 0.3817 Acc: 0.8547

Training complete in 3m 50s
Best test Loss: 0.381680


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0576 Acc: 0.4963
test Loss: 1.0286 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0215 Acc: 0.5027
test Loss: 0.9691 Acc: 0.5359
Epoch 2/9
----------
synthetic_train Loss: 0.9940 Acc: 0.5186
test Loss: 0.9591 Acc: 0.5555
Epoch 3/9
----------
synthetic_train Loss: 0.9502 Acc: 0.5518
test Loss: 0.8278 Acc: 0.6453
Epoch 4/9
----------
synthetic_train Loss: 0.9097 Acc: 0.5732
test Loss: 0.7530 Acc: 0.6758
Epoch 5/9
----------
synthetic_train Loss: 0.8347 Acc: 0.6070
test Loss: 0.8020 Acc: 0.6703
Epoch 6/9
----------
synthetic_train Loss: 0.7771 Acc: 0.6480
test Loss: 0.4899 Acc: 0.8109
Epoch 7/9
----------
synthetic_train Loss: 0.7315 Acc: 0.6652
test Loss: 0.3783 Acc: 0.8664
Epoch 8/9
----------
synthetic_train Loss: 0.6814 Acc: 0.6945
test Loss: 0.6786 Acc: 0.7758
Epoch 9/9
----------
synthetic_train Loss: 0.6256 Acc: 0.7207
test Loss: 0.3357 Acc: 0.8852

Training complete in 3m 41s
Best test Loss: 0.335734


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0540 Acc: 0.4986
test Loss: 0.9586 Acc: 0.5078
Epoch 1/9
----------
synthetic_train Loss: 1.0096 Acc: 0.5145
test Loss: 0.9163 Acc: 0.5461
Epoch 2/9
----------
synthetic_train Loss: 0.9827 Acc: 0.5260
test Loss: 0.8665 Acc: 0.5727
Epoch 3/9
----------
synthetic_train Loss: 0.9583 Acc: 0.5389
test Loss: 0.8419 Acc: 0.6211
Epoch 4/9
----------
synthetic_train Loss: 0.9031 Acc: 0.5686
test Loss: 0.6623 Acc: 0.6875
Epoch 5/9
----------
synthetic_train Loss: 0.8453 Acc: 0.6057
test Loss: 0.5142 Acc: 0.7797
Epoch 6/9
----------
synthetic_train Loss: 0.7788 Acc: 0.6371
test Loss: 0.4487 Acc: 0.8086
Epoch 7/9
----------
synthetic_train Loss: 0.7556 Acc: 0.6414
test Loss: 0.3631 Acc: 0.8539
Epoch 8/9
----------
synthetic_train Loss: 0.7093 Acc: 0.6689
test Loss: 0.3384 Acc: 0.8625
Epoch 9/9
----------
synthetic_train Loss: 0.6806 Acc: 0.6824
test Loss: 0.2919 Acc: 0.8898

Training complete in 3m 44s
Best test Loss: 0.291883
